In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/Shareddrives/NLP Project'

Mounted at /content/drive
/content/drive/Shareddrives/NLP Project


In [ ]:
import os
from eval_script import RecordTrack1
from nltk.tokenize import WordPunctTokenizer
import json
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [ ]:
def process_n2c2_files_task3(base_dir, data_split, tokenizer, annotation_type, test_ann_dir=None):
    '''
    base_dir: (str) base directory (abs or rel to current)
    data_split: (str) train, test, or dev
    tokenizer: tokenizer instance of nltk library
    '''
    text_dir = os.path.join(base_dir, data_split)
    # If we are testing, use the annotations generated in Task 2 Event
    if data_split =="test":
        ann_dir = os.path.join(base_dir, 'output', test_ann_dir)
    else:
        ann_dir = text_dir
    root_names = [file.replace(".ann", "") for file in 
                  os.listdir(ann_dir) if ".ann" in file]
    counts = {}
    json_output = []
    sent_tokenizer = PunktSentenceTokenizer()
    all_mentions = []
    for root_name in root_names:
        ann_file = os.path.join(ann_dir, root_name + ".ann")
        txt_file = os.path.join(text_dir, root_name + ".txt")
        txt = open(txt_file, "r")
        doc_text = txt.read()
        sentences = sent_tokenizer.tokenize(doc_text)
        spans = sent_tokenizer.span_tokenize(doc_text)
        # Process annotation file for same file
        # Only keep medical mentions for now; sort by start pos
        annotation = RecordTrack1(ann_file)
        all_ann = annotation.annotations['attributes'].values()
        anns = sorted([ann for ann in all_ann if ann.rtype == annotation_type],
                       key = lambda item: item.arg.start)
        

        all_mentions.extend(anns)
        for sent_start, sentence in enumerate(sentences):
            start_idx = spans[sent_start][0]
            end_idx = spans[sent_start][1]
            for ann in anns:
                if ann.arg.start >= start_idx and ann.arg.start <= end_idx:
                    if (sent_start > 0) and (sent_start < (len(sentences) - 1)):
                        # all 3 sentences
                        text = sentences[sent_start - 1] + sentence + sentences[sent_start + 1]
                    elif sent_start > 0:
                        # prev_sentence + sentence
                        text = sentences[sent_start - 1] + sentence
                    elif sent_start < (len(sentences) - 1):
                        # sentence + next_sentence
                        text = sentence + sentences[sent_start + 1]
                    label = ann.rval
                    if label not in counts:
                        counts[label] = 0
                    counts[label] += 1
                    json_output.append({"text": text, 
                                        "label": label, 
                                        "note_id": root_name, 
                                        "tid": ann.arg.rid, 
                                        "rid": ann.rid}) 
        assert len(all_mentions) == len(json_output)
    print(counts)
    print(json_output)
    # print(len(json_output))
    # print(len(all_med_mentions))
    with open(f"{base_dir}/input/context_input/{annotation_type.lower()}_{data_split}.json", 'w') as fp:
        fp.write('\n'.join(json.dumps(i) for i in json_output) +'\n')

In [ ]:
base_dir = 'data/split_data'
test_ann_dir = 'event_predicted_annotations_from_ner'
tokenizer = WordPunctTokenizer()

for data_split in ['train', 'dev']:
  print(data_split)
  for ann_type in ["Certainty", "Negation", "Temporality", "Action", "Actor"]:
    print(ann_type)
    process_n2c2_files_task3(base_dir, data_split, tokenizer, ann_type, test_ann_dir)


train
Certainty
{'Certain': 897, 'Hypothetical': 97, 'Conditional': 74, 'Unknown': 2}
[{'text': 'It was felt that her low hematocrit was due to both blood loss from\n\nPersistent severe menorrhagia as well as a dilutional component from fluid overload.She was electively \n\ntransfused 2 units of packed cells on 9/05/86 while on dialysis utilizing her revised A-V fistulae.The rest of her hospital course was uneventful.', 'label': 'Certain', 'note_id': '326-03', 'tid': 'E3', 'rid': 'A1'}, {'text': 'However, per Mr.Vaughn, he arrived to the appointment only with the expectation of discussing his depression and refilling his Celexa, which he has been unable to take for the past 2 weeks, due to an expired prescription.Dr.', 'label': 'Certain', 'note_id': '322-04', 'tid': 'E1', 'rid': 'A1'}, {'text': 'IDDM2:  Diagnosed at age 18.Was previously on oral hypoglycemics, but now on insulin.When pt had lost weight and down to 210 lbs, he did not require insulin.', 'label': 'Certain', 'note_id': '3